In [ ]:
!pip install transformers

In [ ]:
import numpy as np
import pandas as pd
from torch.nn import functional as F
from transformers import BertTokenizer, BertForMaskedLM
import torch


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/df_15000.csv')


# Get the lists of sentences and their labels.
sentences = df['QUESTION'].values
sentences1 = df['QUESTION'].values
labels = df['ANSWER'].values

combine = ''
combine1 = ''


for i in range(len(sentences)):
    # print('Encoding the tokens..',cnt)
    sent = sentences[i].replace('[BLANK]',labels[i],1)
    combine1 = combine1+"\n\n"+sent

    sent1 = sentences[i].replace('[BLANK]','BLANK',1)
    combine = combine+"\n\n"+sent1

In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/Tdf_5000.csv')

In [ ]:
combine[:1000]

"\n\nIncreased content consumption on mobile devices may further drive distributors such as cable and telecom operators to release over-the-top mobile video offerings, while shifting ad budgets to the Internet. Smartphone and tablet BLANK reached 33% of total online video plays in the fourth quarter, according to Ooyala, doubling from 17% in 4Q13. A number of content distributors have responded to this trend, with Verizon announcing a mobile video service to debut this summer.\n\nNational Australia Bank's earnings prospects will rely on an ongoing strength of business and corporate banking, which jointly contributed 65% of its 1H pretax profit. These units showed stable net interest margins, BLANK a weakness in consumer banking. Market income also supports its revenue, as market volatility fuels demand for risk management and treasury business. Reduced credit costs bode well for its earnings too. Separately, NAB and peers' profit will suffer from the proposed 6-bp levy on liabilities.\

In [ ]:
inputs1 = tokenizer(combine1, return_tensors='pt', max_length=512, truncation=True, padding='max_length')


In [ ]:
inputs1.keys()


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
inputs = tokenizer(combine, return_tensors='pt', max_length=512, truncation=True, padding='max_length')


In [ ]:
inputs.keys()


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
inputs['labels'] = inputs1.input_ids.detach().clone()
inputs

{'input_ids': tensor([[  101,  3445,  4180,  8381,  2006,  4684,  5733,  2089,  2582,  3298,
         22495,  2107,  2004,  5830,  1998, 18126,  9224,  2000,  2713,  2058,
          1011,  1996,  1011,  2327,  4684,  2678, 14927,  1010,  2096,  9564,
          4748, 26178,  2000,  1996,  4274,  1012, 26381,  1998, 13855,  8744,
          2584,  3943,  1003,  1997,  2561,  3784,  2678,  3248,  1999,  1996,
          2959,  4284,  1010,  2429,  2000,  1051, 18232,  2721,  1010, 19383,
          2013,  2459,  1003,  1999,  1018,  4160, 17134,  1012,  1037,  2193,
          1997,  4180, 22495,  2031,  5838,  2000,  2023,  9874,  1010,  2007,
          2310, 21885,  2239, 13856,  1037,  4684,  2678,  2326,  2000,  2834,
          2023,  2621,  1012,  2120,  2660,  2924,  1005,  1055, 16565, 16746,
          2097, 11160,  2006,  2019,  7552,  3997,  1997,  2449,  1998,  5971,
          8169,  1010,  2029, 10776,  5201,  3515,  1003,  1997,  2049,  1015,
          2232,  3653,  2696,  2595,  

In [ ]:
# create mask array
mask_arr = (inputs.input_ids == 8744) * (inputs.input_ids != 101) * (inputs.input_ids != 102) * (inputs.input_ids != 0)
print(mask_arr[0][39:50])

tensor([ True, False, False, False, False, False, False, False, False, False,
        False])


In [ ]:
inputs.input_ids.shape

torch.Size([1, 512])

In [ ]:
# create selection from mask_arr
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

selection

[[39, 134, 268, 319, 428, 482]]

In [ ]:
# apply selection index to inputs.input_ids, adding MASK tokens

for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

inputs.input_ids

tensor([[  101,  3445,  4180,  8381,  2006,  4684,  5733,  2089,  2582,  3298,
         22495,  2107,  2004,  5830,  1998, 18126,  9224,  2000,  2713,  2058,
          1011,  1996,  1011,  2327,  4684,  2678, 14927,  1010,  2096,  9564,
          4748, 26178,  2000,  1996,  4274,  1012, 26381,  1998, 13855,   103,
          2584,  3943,  1003,  1997,  2561,  3784,  2678,  3248,  1999,  1996,
          2959,  4284,  1010,  2429,  2000,  1051, 18232,  2721,  1010, 19383,
          2013,  2459,  1003,  1999,  1018,  4160, 17134,  1012,  1037,  2193,
          1997,  4180, 22495,  2031,  5838,  2000,  2023,  9874,  1010,  2007,
          2310, 21885,  2239, 13856,  1037,  4684,  2678,  2326,  2000,  2834,
          2023,  2621,  1012,  2120,  2660,  2924,  1005,  1055, 16565, 16746,
          2097, 11160,  2006,  2019,  7552,  3997,  1997,  2449,  1998,  5971,
          8169,  1010,  2029, 10776,  5201,  3515,  1003,  1997,  2049,  1015,
          2232,  3653,  2696,  2595,  5618,  1012,  

In [ ]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
dataset = MeditationsDataset(inputs)


In [ ]:
loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)
# activate training mode
model.train()

In [ ]:
from transformers import AdamW
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-5)

In [ ]:
from tqdm import tqdm  # for our progress bar

epochs = 40

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        logits = outputs.logits


        

        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 39: 100%|██████████| 1/1 [00:08<00:00,  8.60s/it, loss=0.278]


## testing model

In [ ]:
df_test

,Unnamed: 0,INDEX,QUESTION,ANSWER
0,15000,15000,While construction has begun on the Nord Strea...,grant
1,15001,15001,"A case against 47 people, including OMV execut...",passed
2,15002,15002,Property insurers are positioning themselves t...,tied
3,15003,15003,Low-cost European airlines averaged 9% capacit...,garner
4,15004,15004,Some of the junket operators integral to Macau...,plunged
...,...,...,...,...
4995,19995,19995,"Manufacturing PMIs for new orders, a bellwethe...",dropped
4996,19996,19996,Approaching the fourth anniversary of Dodd-Fra...,vote
4997,19997,19997,A 2015 spinoff of Dupont's TiO2 business is on...,fell
4998,19998,19998,Margins for Texas merchant generators Calpine ...,threatens


In [ ]:
split_arr = []
for i in range(30):
  text = df_test['QUESTION'][i]
  split_arr.append(text.split("[BLANK]"))

df_T = df_test[:30]
df_split = pd.DataFrame(split_arr,columns=['split1','split2'])
df_T = pd.concat([df_T,df_split],axis=1)
df_T = df_T[['QUESTION','ANSWER','split1','split2']]
df_T

,QUESTION,ANSWER,split1,split2
0,While construction has begun on the Nord Strea...,grant,While construction has begun on the Nord Strea...,the necessary approvals. Nord Stream 2 is als...
1,"A case against 47 people, including OMV execut...",passed,"A case against 47 people, including OMV execut...","on to the buyer. Yet, the allegations could a..."
2,Property insurers are positioning themselves t...,tied,Property insurers are positioning themselves t...,to an unsustainable system. Modest reforms pr...
3,Low-cost European airlines averaged 9% capacit...,garner,Low-cost European airlines averaged 9% capacit...,"higher yields. EasyJet added only 2.5%, compa..."
4,Some of the junket operators integral to Macau...,plunged,Some of the junket operators integral to Macau...,42% in the first quarter. With close relation...
5,Recovering state budgets are supporting road-r...,led,Recovering state budgets are supporting road-r...,"by pavement spending, up 8.5%. Pavement const..."
6,An increase in the number of U.S. hotel rooms ...,experience,An increase in the number of U.S. hotel rooms ...,"this decline first, since expansion there has..."
7,"BlackRock, Prudential Financial and other mort...",narrowing,"BlackRock, Prudential Financial and other mort...",the $98 billion lawsuit against the bank in i...
8,Legislation that would give the U.S. Departmen...,controls,Legislation that would give the U.S. Departmen...,"either chamber of Congress, this measure favo..."
9,Neither side of the legal challenge to the EPA...,committed,Neither side of the legal challenge to the EPA...,to seeing the process through to the end. Law...


In [ ]:
labels = []
for i in range(30):
  labels_one_sentence = []
  text = df_T['split1'][i] + tokenizer.mask_token + df_T['split2'][i]

  input = tokenizer.encode_plus(text, return_tensors = "pt")
  mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)



  output = model(**input)
  logits = output.logits

  softmax = F.softmax(logits, dim = -1)
  mask_word = softmax[0, mask_index, :]

  top_10 = torch.topk(mask_word, 5, dim = 1)[1][0]
  for token in top_10:
    word = tokenizer.decode([token])
    labels_one_sentence.append(word)

  labels.append(labels_one_sentence)

In [ ]:
df_label = pd.DataFrame(labels)


In [ ]:
df_answer = pd.concat([df_T['ANSWER'],df_label],axis=1)
df_answer

,ANSWER,0,1,2,3,4
0,grant,obtain,submit,get,complete,secure
1,passed,passed,passing,pass,passes,left
2,tied,vulnerable,subject,exposed,due,susceptible
3,garner,deliver,achieve,obtain,offer,get
4,plunged,reached,increased,was,grew,rose
5,led,followed,led,fueled,driven,helped
6,experience,see,face,suffer,trend,have
7,narrowing,ending,filing,dismissing,in,dismissed
8,controls,in,of,from,is,and
9,committed,committed,loyal,leading,accustomed,devoted
